<a href="https://colab.research.google.com/github/DeepPersona/Workshop_JMSB-1/blob/main/Sample1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Replace 'your_username' and 'project_repo' with your actual information
!git clone https://github.com/DeepPersona/Workshop_JMSB-1.git

Cloning into 'Workshop_JMSB-1'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), 814.97 KiB | 10.72 MiB/s, done.


In [3]:
# Change directory to the cloned repository
%cd Workshop_JMSB-1

/content/Workshop_JMSB-1


In [4]:
import pandas as pd

# Use the relative path to your CSV file from the root of your repo
# Modify 'your_csv_file.csv' to match the name of your CSV
csv_file_path = 'rESPRESSO-jul.csv'

try:
    df = pd.read_csv(csv_file_path)
    print("CSV loaded successfully. Here are the first 5 rows:")
    print(df.head())

    # --- Start Data Manipulation ---

    # You can now perform your data analysis here.
    # For example, display summary statistics:
    print("\nDataFrame Summary:")
    print(df.describe())

except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found. Please check the name and path.")

CSV loaded successfully. Here are the first 5 rows:
   score        date                                              title  \
0     75  2025-07-01                             My super budget set up   
1      1  2025-07-01  I ordered a Gaggia CP E24 from Whole Latte Lov...   
2      3  2025-07-01                                Is this model good?   
3      1  2025-07-01             Give me critique with my milk frothing   
4      1  2025-07-01                         Shot pulling for a newbie?   

                   author                                          permalink  \
0      u/Purple-Depth9610  https://www.reddit.com/r/espresso/comments/1lo...   
1             u/REDBOSS27  https://www.reddit.com/r/espresso/comments/1lo...   
2          u/boomsauce120  https://www.reddit.com/r/espresso/comments/1lo...   
3  u/Human_Dragonfruit236  https://www.reddit.com/r/espresso/comments/1lo...   
4             u/TDSRage97  https://www.reddit.com/r/espresso/comments/1lo...   

                

In [5]:
!pip install openai
import pandas as pd
from openai import OpenAI


In [8]:
import os
from google.colab import userdata

# Store your API key as a secret in Colab and retrieve it here
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
except:
    # If not using Colab secrets, you can store it as an environment variable
    openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OpenAI API key not found. Please set it up in Colab secrets or environment variables.")

client = OpenAI(api_key=openai_api_key)


In [9]:
import pandas as pd
from openai import OpenAI
import os
from google.colab import userdata

# Set up your OpenAI API key from Colab secrets
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
except:
    openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OpenAI API key not found. Please set it up in Colab secrets or environment variables.")

client = OpenAI(api_key=openai_api_key)

def summarize_csv_column_with_openai(
    df: pd.DataFrame,
    column_name: str,
    num_samples: int = 5,
    model: str = "gpt-3.5-turbo",
    temperature: float = 0.7,
    max_tokens: int = 150
):
    """
    Summarizes a specified column of a DataFrame using the OpenAI API.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column to summarize.
        num_samples (int): The number of samples to send to the API.
        model (str): The OpenAI model to use for the analysis.
        temperature (float): The sampling temperature to use (0 to 2).
                             Higher values mean more creative output.
        max_tokens (int): The maximum number of tokens to generate.

    Returns:
        str: The summaries generated by the OpenAI model.
    """
    if column_name not in df.columns:
        return f"Analysis failed: Column '{column_name}' not found."

    # 1. Prepare data by sampling
    cleaned_df = df.dropna(subset=[column_name])
    if 'text' in cleaned_df.columns:
        cleaned_df = cleaned_df[cleaned_df[column_name].str.strip() != '[removed]']

    if len(cleaned_df) == 0:
        return "No valid text data to summarize."

    sample_texts = cleaned_df[column_name].sample(n=min(num_samples, len(cleaned_df)), random_state=42).tolist()

    # 2. Format the sample data for the prompt
    formatted_texts = "\n\n".join([f"Post {i+1}:\n{text}" for i, text in enumerate(sample_texts)])

    # 3. Construct the prompt
    system_prompt = (
        "You are a helpful assistant that summarizes text from social media posts. "
        "Your response should be concise and provide a high-level summary of each post."
    )
    user_prompt = (
        f"Please provide a summary for each of the following posts from the '{column_name}' column. "
        f"Present each summary in a clear and readable format.\n\n{formatted_texts}"
    )

    print("Sending request to OpenAI...")

    # 4. Call the OpenAI API with the specified variables
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=temperature,
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred with the OpenAI API call: {e}"



In [ ]:
# Assuming you have already run the code to set up your OpenAI API client
# and have the `summarize_csv_column_with_openai` function defined.

import pandas as pd
from google.colab import files
import io

# 1. Upload the CSV file
print("Please upload your CSV file named 'rESPRESSO-jul.csv'")
uploaded = files.upload()

# 2. Read the uploaded CSV file into a pandas DataFrame
csv_filename = 'rESPRESSO-jul.csv'
try:
    if csv_filename in uploaded:
        df = pd.read_csv(io.StringIO(uploaded[csv_filename].decode('utf-8')))
        print("CSV loaded successfully.")
    else:
        raise FileNotFoundError(f"{csv_filename} was not found among the uploaded files.")
except Exception as e:
    print(f"Error loading CSV file: {e}")
    # Exit or handle error as needed
    df = pd.DataFrame() # Create an empty df to prevent further errors

# 3. Select only the first 5 rows for analysis
if not df.empty:
    df_first_5 = df.head(5)

    # 4. Define LLM variables for customization
    llm_temperature = 0.5
    llm_max_tokens = 200

    # 5. Call the function to summarize the 'text' column from the first 5 rows
    summary_result_text = summarize_csv_column_with_openai(
        df_first_5,
        column_name='text',
        num_samples=5, # We already limited to 5 rows, but this keeps it explicit
        temperature=llm_temperature,
        max_tokens=llm_max_tokens
    )

    print("\n--- Summary for the first 5 posts from the 'text' column ---")
    print(f"Temperature: {llm_temperature}, Max Tokens: {llm_max_tokens}\n")
    print(summary_result_text)

    # 6. Call the function to summarize the 'title' column from the first 5 rows
    summary_result_title = summarize_csv_column_with_openai(
        df_first_5,
        column_name='title',
        num_samples=5,
        temperature=llm_temperature,
        max_tokens=llm_max_tokens
    )

    print("\n--- Summary for the first 5 posts from the 'title' column ---")
    print(f"Temperature: {llm_temperature}, Max Tokens: {llm_max_tokens}\n")
    print(summary_result_title)
else:
    print("DataFrame is empty. Cannot perform analysis.")



Please upload your CSV file named 'rESPRESSO-jul.csv'
